# Análise dos Resultados de Rendimento do Ensino em tempos de Pandemia (2018 - 2022)

## Objetivo

O objetivo deste notebook é demonstrar como podemos utilizar o Fluxo de Trabalho de um Cientista de Dados para resolver problemas e entregar soluções orientadas a dados.

O Fluxo de Trabalho é composto de 6 fases:
1) Entender o problema
2) Coletar os dados
3) Transformar os dados
4) Explorar os dados
5) Comunicar resultados
6) Coletar feedback

![Ciclo de vida em Ciência de Dados](imagens/ciclo_vida_ds.png)

Iremos pegar um problema para o qual gostaríamos de encontrar uma solução suportada por dados e percorrer cada uma das fases (1 a 4) e mostrar o que um Cientista de Dados normalmente faz em cada uma delas.

## 0 - O Problema

Imagine a seguinte situação: você é um(a) Cientista de Dados que recebeu o seguinte questionamento: 

> Muito se tem falado sobre como o ensino remoto durante o tempo de pandemia deixou a desejar em termos de qualidade, principalmente devido as grandes desigualdades social, tecnológica e econômica - que já existiam mas que o ensino remoto tornou muito mais aparente  - mas como podemos apoiar essa afirmação com dados?



## 1 - Entendendo o Problema

Traduzindo o problema descrito acima: precisamos encontrar dados para validar a hipótese de que o modelo de educação remota utilizado durante a pandemia não foi adequado e portanto, os alunos tiveram o seu aprendizado prejudicado.

Consultando na Internet, podemos encontrar várias reportagens e artigos científicos que confirmam essa afimação e muitos deles já utilizam dados. E também, este é um problema muito grande para ser atacado de maneira simplista. Então, baseado no problema, vamos formular uma hipótese e vamos encontrar os dados que nos ajudem a comprovar ou descartar essa hipótese.

### Hipótese

> A baixa qualidade do ensino remoto que ocorreu durante a pandemia, devido a falta de preparo adequada (situação de emergência) e as desigualdades social, tecnológica e econômica já vigentes no cenário da Educação Brasileira, levou a uma piora do desempenho dos alunos em relação a anos anteriores.

Parece bom, não é? Agora, temos um problema bem claro e dimensionado para tentar responder.

## 2 - Identificando e Coletando os dados

Agora, precisamos identificar que dados podemos utilizar e onde encontra-los.

Da maneira que formulamos nossa hipótese, estamos interessados em avaliar o desempenho dos alunos e utilizar essa medida de desempenho para verificar se nos anos de pandemia, o desempenho foi menor do que nos anos anteriores. E qual seria a medida de desempenho dos alunos que poderíamos utilizar? Vamos descobrir em seguida.

Definido os tipos de dados de que precisamos, vamos agora descobrir onde pega-los: o site oficial para essas informações é o site do INEP - (https://www.gov.br/inep/pt-br)[https://www.gov.br/inep/pt-br]. Ao navegarmos no site, podemos chegar até as (Taxas de Rendimento)[https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/taxas-de-rendimento-escolar].

Ali, teremos a oportunidade de realizar o download das informações, um arquivo por ano. Iremos realizar o download dos anos de 2018 a 2022, para pegar um ano anterior ao inicio da pandemia e até o último ano da pandemia.

![Microdados do ENEM](imagens/site_inep_rend.png)

## 3 - Preparando os Dados

Agora que temos nossos dados, precisamos prepara-los para serem utilizados.

Os arquivos de dados das taxas de rendimento que fizemos download são arquivos ZIP que contém diversos arquivos, como mostrado abaixo.

![Exemplo de arquivo de microdados](imagens/exemplo_arquivo_rend.png)

Como se tratam de arquivos mantidos através da Lei de Acesso a Dados, eles estão bem organizados. Em muitos casos, não temos tanta sorte.

Neste caso, iremos utilizar o arquivo TX_REND_MUNICIPIOS_`<ANO>`.xlsx como nossa fonte de dados.

Como são arquivos excel, iremos transforma-los em arquivos CSV, de forma que poderemos juntar todos estes arquivos em um único arquivo Excel para análise.


### 3.1 Limpando os dados e Convertendo

Para podermos utilizar os arquivos, vamos executar os seguintes passos:
1) Abrir o arquivo Excel
2) Remover as 8 primeiras linhas
3) Salvar como arquivo do tipo CSV

Executados esses passos para todos os arquivos que baixamos. Veja o resultado abaixo.

![Dados convertidos](imagens/dados_convertidos.png)

### 3.2 Código

#### Importando Bibliotecas

In [1]:
import pandas as pd
import os

#### Variáveis de Configuração

In [2]:
# Definindo as pastas para salvar os arquivos originais e o arquivo processado
pasta_rend = "dados-rendimento"
pasta_output = "output"
arquivos_excluidos = [".DS_Store","placeholder.txt"]

O nosso código será muito similar ao pseucódigo abaixo:

```python
para cada arquivo na pasta_enem
  carrega o arquivo em memória
  seleciona apenas as colunas de interesse
  adiciona a lista

realiza o merge
salva como CSV
```

In [3]:
arquivos = os.listdir(pasta_rend)
lista_dfs = []

for arquivo in arquivos:
  if arquivo not in arquivos_excluidos:
    print(arquivo)
    raw_df = pd.read_csv(os.path.join(pasta_rend, arquivo), encoding="utf-8", encoding_errors="replace", sep=";")
    print("Aplicando correções...")
    raw_df.replace(["--"],[0], inplace=True)
        
    lista_dfs.append(raw_df)

print("Concatenando dataframes...")
final_df = pd.concat(lista_dfs)
print("Salvando o arquivo final...")
final_df.to_excel(os.path.join(pasta_output, "taxa_rendimento_full.xlsx"))

tx_rend_municipios_2019.csv
Aplicando correções...
TX_REND_MUNICIPIOS_2018.csv
Aplicando correções...
tx_rend_municipios_2022.csv
Aplicando correções...
tx_rend_municipios_2020.csv
Aplicando correções...
tx_rend_municipios_2021.csv
Aplicando correções...
Concatenando dataframes...
Salvando o arquivo final...


O erro acima indica que nosso computador tem um problema em manter todos os dados na memória. Para contornar esse problema, iremos mandar todos os dados para um banco de dados, assim evitando manter tudo na memória. Vamos as modificações no código.